In [142]:
print(__doc__)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
pd.options.display.float_format = "{:.4f}".format
pd.set_option("display.max_rows", None, "display.max_columns", None)

Automatically created module for IPython interactive environment


In [143]:
path = r'/Users/wenxuanhuang/Documents/Repo/ML-for-COVID-19-dataset/result_log/af_test/09081220_EQODS_NF(200,10)'
all_files = sorted(glob.glob(os.path.join(path, "*.csv")))

print(all_files[3])

min_abs = []
idx_min_abs = []
min_diff = []
idx_min_diff = []

for f in all_files:
    df = pd.read_csv(f)
    df = df.rename(columns={"# x_axis": "axis", " ur_accuracies": "ur_accuracies", " ur_fairness": "ur_fairness", " rs_accuracies": "rs_accuracies", " rs_fairness": "rs_fairness", " nonal_accuracies": "nonal_accuracies", " nonal_fairness": "nonal_fairness"})
    min_abs = np.append(min_abs, df['ur_fairness'].min())
    # print(min_abs)
    idx_min_abs = np.append(idx_min_abs, df['ur_fairness'].idxmin())
    # print(idx_min_abs)
    df_diff = df.diff()
    min_diff = np.append(min_diff, df_diff['ur_fairness'].min())
    # print(min_diff)
    idx_min_diff = np.append(idx_min_diff, df_diff['ur_fairness'].idxmin())
    # print(idx_min_diff)

df_overview = pd.DataFrame(np.hstack((min_abs[:,None], idx_min_abs[:,None], min_diff[:,None], idx_min_diff[:,None])))

print(df_overview)
candidate_size = int(round(len(df_overview)/2))
idx_diff = abs(df_overview[1]-df_overview[3])
idx_sort = idx_diff.argsort()  
idx_min_diff_top = df_overview.nsmallest(candidate_size, 2, keep='all').index  # index of 5 largest fairness reduction

ideal_trade_off = []
ideal_budget = []

for idx in idx_sort[0:candidate_size]:  # top half with smallest drop index and min index difference
    if idx in idx_min_diff_top.unique():  # is it one of the largest impact on fairness reduction
        df_min = pd.read_csv(all_files[idx])
        df_min = df_min.rename(columns={"# x_axis": "axis", " ur_accuracies": "ur_accuracies", " ur_fairness": "ur_fairness", " rs_accuracies": "rs_accuracies", " rs_fairness": "rs_fairness", " nonal_accuracies": "nonal_accuracies", " nonal_fairness": "nonal_fairness"})
        
        if df_min['ur_fairness'].min()-df_min['rs_fairness'].min()<0:
            # print(df_min)
            e_trade_off = idx/10
            ideal_trade_off = np.append(ideal_trade_off, e_trade_off)
            ideal_budget = np.append(ideal_budget, df_min['axis'][df_min['ur_fairness'].idxmin()])

print(ideal_trade_off, ideal_budget)

/Users/wenxuanhuang/Documents/Repo/ML-for-COVID-19-dataset/result_log/af_test/09081220_EQODS_NF(200,10)/3.Result.csv
       0       1       2       3
0 0.0135  2.0000 -0.0361 18.0000
1 0.0147  0.0000 -0.0298  8.0000
2 0.0584  5.0000 -0.0518  4.0000
3 0.0780  2.0000 -0.0421  2.0000
4 0.0060  3.0000 -0.0335 12.0000
5 0.0147  2.0000 -0.1107  6.0000
6 0.0135  0.0000 -0.0150  6.0000
7 0.0885 11.0000 -0.0346  9.0000
8 0.0216  1.0000 -0.0347  1.0000
9 0.0406  2.0000 -0.0329  6.0000
[0.8 0.5] [20. 30.]
